In [ ]:
from gettext import install
import pandas as pd
import numpy as np    
from sklearn.model_selection import train_test_split     
from sklearn import metrics 
from sklearn.preprocessing import OneHotEncoder   
from sklearn.preprocessing import StandardScaler    
from sklearn.preprocessing import LabelEncoder  
from sklearn.metrics import f1_score  
from sklearn.pipeline import Pipeline  
from sklearn.impute import SimpleImputer  
from sklearn.compose import ColumnTransformer
import multiprocessing
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupKFold
from sklearn.utils import resample
%pip install imbalanced-learn
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.impute import SimpleImputer


Note: you may need to restart the kernel to use updated packages.


In [2]:
trc=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAIN_CATEGORICAL_METADATA.xlsx')  
trq=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAIN_QUANTITATIVE_METADATA.xlsx')    
trf=pd.read_csv(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv')
trs=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAINING_SOLUTIONS.xlsx')  
tsc=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Test\TEST_CATEGORICAL.xlsx')
tsq=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TEST_QUANTITATIVE_METADATA.xlsx')    
tsf=pd.read_csv(r'C:\Users\Family\Desktop\WIDS-Com\Test\TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
sub=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Submission\SAMPLE_SUBMISSION.xlsx')    
dic=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Submission\Data Dictionary.xlsx')

Data Merging 

In [3]:
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')  

Data Preprocessing

In [4]:
train.describe()

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,...,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn,ADHD_Outcome,Sex_F
count,1213.000000,1213.000000,1202.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,...,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000
mean,2017.652102,2.014839,0.424293,2.080791,2.288541,17.641385,24.892828,14.122012,24.719703,58.876917,...,-0.004378,0.001610,0.115171,0.049984,0.058144,0.093527,0.089403,0.128946,0.685078,0.342951
std,1.122522,1.135147,0.687470,3.164636,0.758348,4.004639,17.025899,7.201023,17.171113,49.855501,...,0.042900,0.047424,0.057128,0.051664,0.057674,0.054594,0.058036,0.058490,0.464676,0.474891
min,2015.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-100.000000,...,-0.178688,-0.138048,-0.070094,-0.153540,-0.131455,-0.085566,-0.204160,-0.083077,0.000000,0.000000
25%,2017.000000,1.000000,0.000000,0.000000,2.000000,15.000000,0.000000,12.000000,5.000000,45.600000,...,-0.033798,-0.030132,0.080291,0.015827,0.022316,0.059621,0.053224,0.090459,0.000000,0.000000
50%,2018.000000,1.000000,0.000000,0.000000,2.000000,18.000000,30.000000,18.000000,30.000000,76.670000,...,-0.003724,0.000990,0.113640,0.052705,0.059151,0.093397,0.088612,0.127913,1.000000,0.000000
75%,2019.000000,3.000000,1.000000,2.000000,3.000000,21.000000,40.000000,21.000000,40.000000,93.400000,...,0.024007,0.032268,0.150524,0.082526,0.095192,0.127144,0.127613,0.166523,1.000000,1.000000
max,2020.000000,4.000000,3.000000,11.000000,4.000000,21.000000,45.000000,21.000000,45.000000,100.000000,...,0.180562,0.192015,0.375635,0.228748,0.322084,0.348153,0.267162,0.414304,1.000000,1.000000


In [5]:
c_train = train.copy()
c_test = test.copy()    


In [6]:
X = c_train.drop(['participant_id', 'ADHD_Outcome', 'Sex_F'], axis=1, errors='ignore')
y_adhd = c_train[['ADHD_Outcome']]
y_sex = c_train[['Sex_F']]  
y_tr = pd.merge(y_adhd , y_sex, left_index=True, right_index=True)   


In [7]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(exclude=['object']).columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('imputer', SimpleImputer(strategy='median'))   
])


categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='constant', fill_value='most_frequent')) 
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X_tr = preprocessor.fit_transform(X)
X_ts = preprocessor.transform(test.drop('participant_id', axis=1))
y_tr = preprocessor.fit_transform(X)

In [8]:
model = RandomForestClassifier()
model.fit(X_tr, y_adhd)

X_tr_df = pd.DataFrame(X_tr, columns=numerical_features + categorical_features)

feature_importance = model.feature_importances_
important_adhd = pd.DataFrame({"Feature": X_tr_df.columns, "Importance": feature_importance})
important_adhd.sort_values("Importance", ascending=False)

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Feature,Importance
22,SDQ_SDQ_Hyperactivity,0.011168
20,SDQ_SDQ_Externalizing,0.008475
18,SDQ_SDQ_Difficulties_Total,0.004911
21,SDQ_SDQ_Generating_Impact,0.003309
17,SDQ_SDQ_Conduct_Problems,0.002360
...,...,...
2517,12throw_181thcolumn,0.000000
9953,58throw_96thcolumn,0.000000
9954,58throw_97thcolumn,0.000000
9956,58throw_99thcolumn,0.000000


In [9]:
model = RandomForestClassifier()
model.fit(X_tr, y_sex)

X_tr_df = pd.DataFrame(X_tr, columns=numerical_features + categorical_features)

feature_importance = model.feature_importances_
important_sex = pd.DataFrame({"Feature": X_tr_df.columns, "Importance": feature_importance})
important_sex.sort_values("Importance", ascending=False)

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Feature,Importance
19739,180throw_183thcolumn,0.000861
6288,34throw_91thcolumn,0.000798
13498,86throw_99thcolumn,0.000792
16472,116throw_148thcolumn,0.000775
1709,8throw_127thcolumn,0.000732
...,...,...
8232,46throw_133thcolumn,0.000000
8233,46throw_134thcolumn,0.000000
8234,46throw_135thcolumn,0.000000
8235,46throw_136thcolumn,0.000000


MIR is imprtant in sex !

In [10]:
X_tr_df = pd.DataFrame(X_tr, columns=numerical_features + categorical_features)
X_ts_df = pd.DataFrame(X_ts, columns=numerical_features + categorical_features)

In [11]:
unimportant_adhd = important_adhd[important_adhd['Importance'] == 0.000]
unimportant_sex = important_sex[important_sex['Importance'] == 0.000]

unimportant_adhd_features = set(unimportant_adhd['Feature'].tolist())
unimportant_sex_features = set(unimportant_sex['Feature'].tolist())

unimportant_features_list = unimportant_adhd_features.intersection(unimportant_sex_features)

X_tr_df = X_tr_df.drop(unimportant_features_list, axis=1)
X_ts_df = X_ts_df.drop(unimportant_features_list, axis=1)



In [12]:
X_tr_df.describe()


,Basic_Demos_Enroll_Year,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,APQ_P_APQ_P_ID,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,...,194throw_197thcolumn,194throw_198thcolumn,194throw_199thcolumn,195throw_196thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,198throw_199thcolumn
count,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,...,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000
mean,0.530420,0.549327,0.794385,0.471617,0.635202,0.841550,0.205936,0.356554,0.230750,0.377865,...,0.499630,0.490022,0.534250,0.484843,0.423127,0.415644,0.532383,0.418043,0.412924,0.426278
std,0.224504,0.381580,0.249278,0.136135,0.134433,0.131684,0.202332,0.193447,0.216755,0.208331,...,0.125669,0.146202,0.156355,0.139304,0.143683,0.128167,0.135143,0.127165,0.125874,0.117595
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.111111,0.728000,0.392857,0.571429,0.766667,0.000000,0.205882,0.100000,0.200000,...,0.421432,0.390725,0.434275,0.393760,0.326957,0.337390,0.443035,0.339048,0.334749,0.348899
50%,0.600000,0.666667,0.883350,0.464286,0.642857,0.866667,0.200000,0.352941,0.200000,0.350000,...,0.496138,0.487402,0.530480,0.482540,0.421247,0.412209,0.539502,0.420263,0.412624,0.424202
75%,0.800000,0.888889,0.967000,0.571429,0.714286,0.933333,0.300000,0.500000,0.400000,0.500000,...,0.577452,0.587945,0.641781,0.577720,0.516012,0.494960,0.617509,0.499731,0.490433,0.501830
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


 Convert the scales of social factors variables

In [13]:
from imblearn.under_sampling import NearMiss

undersampler = NearMiss(version=2, sampling_strategy=1.0)  


X_resampled, y_resampled = undersampler.fit_resample(X_tr_df, y_sex)

train_resampled = pd.DataFrame(X_resampled, columns=X_tr_df.columns)
train_resampled['y_sex'] = y_resampled

print(f"Original size: {len(X_tr_df)}, Resampled size: {len(train_resampled)}")




Original size: 1213, Resampled size: 832


C:\Users\Family\AppData\Local\Temp\ipykernel_10336\2340534110.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_resampled['y_sex'] = y_resampled


In [14]:
y_resampled.describe()

,Sex_F
count,832.000000
mean,0.500000
std,0.500301
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [15]:
y_sex.columns

Index(['Sex_F'], dtype='object')

In [16]:
train_resampled.describe()

,Basic_Demos_Enroll_Year,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,APQ_P_APQ_P_ID,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,...,194throw_198thcolumn,194throw_199thcolumn,195throw_196thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,198throw_199thcolumn,y_sex
count,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,...,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000
mean,0.567548,0.562500,0.801698,0.471626,0.628735,0.840505,0.195673,0.344881,0.229928,0.363702,...,0.486483,0.533442,0.484156,0.422624,0.415050,0.535315,0.419048,0.413532,0.426218,0.500000
std,0.203589,0.379277,0.245127,0.132766,0.130050,0.124295,0.198806,0.190734,0.216816,0.206800,...,0.138672,0.151125,0.132762,0.139270,0.124862,0.132050,0.125925,0.119302,0.111564,0.500301
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.072662,0.037697,0.000000,0.120763,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.111111,0.733350,0.392857,0.535714,0.766667,0.000000,0.205882,0.000000,0.200000,...,0.397596,0.435313,0.396339,0.328350,0.339528,0.449320,0.342281,0.337668,0.351290,0.000000
50%,0.600000,0.666667,0.897350,0.464286,0.642857,0.866667,0.100000,0.323529,0.200000,0.350000,...,0.486121,0.530543,0.481846,0.422295,0.415864,0.542904,0.422765,0.412555,0.428155,0.500000
75%,0.800000,0.888889,0.972350,0.571429,0.714286,0.933333,0.300000,0.470588,0.400000,0.500000,...,0.578037,0.642011,0.571301,0.512107,0.490451,0.616603,0.499042,0.488799,0.499763,1.000000
max,1.000000,1.000000,1.000000,0.857143,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.874354,0.973056,1.000000,0.954007,0.857646,1.000000,0.843098,0.841832,0.806555,1.000000


In [17]:
print(f"Original size: {len(X_tr_df)}, Resampled size: {len(train_resampled)}")

Original size: 1213, Resampled size: 832


feature engineering

In [ ]:
"""X_tr_df["emotional_affectness"] = X_tr_df["SDQ_SDQ_Emotional_Problems"] * X_tr_df["SDQ_SDQ_Internalizing"]
X_tr_df["physical_affectness"] = X_tr_df["SDQ_SDQ_Externalizing"] * X_tr_df["SDQ_SDQ_Hyperactivity"]"""


In [ ]:
"""X_ts_df["emotional_affectness"] = X_ts_df["SDQ_SDQ_Emotional_Problems"] * X_ts_df["SDQ_SDQ_Internalizing"]
X_ts_df["physical_affectness"] = X_ts_df["SDQ_SDQ_Externalizing"] * X_ts_df["SDQ_SDQ_Hyperactivity"]"""

Model Training

xgboost

In [23]:
    # Define groups_train based on a column from your training data
    groups_train = c_train['participant_id']

    group_kfold = GroupKFold(n_splits=5)

    for train_idx, val_idx in group_kfold.split(X_tr_df, y_adhd, groups_train):
        X_train_fold, X_val_fold = X_tr_df.iloc[train_idx], X_tr_df.iloc[val_idx]
        y_train_fold_adhd, y_val_fold_adhd = y_adhd.iloc[train_idx], y_adhd.iloc[val_idx]
        y_train_fold_sex, y_val_fold_sex = y_sex.iloc[train_idx], y_sex.iloc[val_idx]
 

In [ ]:
X_train_fold.describe()

,Basic_Demos_Enroll_Year,Barratt_Barratt_P1_Occ,APQ_P_APQ_P_CP,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,...,193throw_198thcolumn,193throw_199thcolumn,194throw_196thcolumn,194throw_198thcolumn,194throw_199thcolumn,195throw_196thcolumn,195throw_198thcolumn,196throw_197thcolumn,197throw_198thcolumn,198throw_199thcolumn
count,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,...,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000
mean,0.531205,0.560018,0.315997,0.635795,0.443037,0.841813,0.206694,0.353759,0.226880,0.377652,...,0.448148,0.540032,0.437927,0.488797,0.539328,0.483489,0.485796,0.415226,0.414101,0.426752
std,0.225682,0.373989,0.114273,0.133660,0.146744,0.132714,0.204149,0.192977,0.216172,0.207029,...,0.126386,0.142451,0.143228,0.144483,0.155939,0.139586,0.119773,0.128849,0.127392,0.116548
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.072662,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.111111,0.250000,0.571429,0.351351,0.766667,0.000000,0.205882,0.000000,0.250000,...,0.361680,0.443107,0.351371,0.390663,0.440421,0.391345,0.410002,0.340210,0.333746,0.351326
50%,0.600000,0.666667,0.250000,0.642857,0.432432,0.866667,0.200000,0.352941,0.200000,0.350000,...,0.450388,0.543298,0.436301,0.487943,0.533388,0.481241,0.489476,0.412530,0.412486,0.424202
75%,0.800000,0.888889,0.333333,0.714286,0.513514,0.933333,0.300000,0.500000,0.400000,0.500000,...,0.531746,0.625897,0.536429,0.583647,0.645975,0.578549,0.564509,0.491761,0.490295,0.500705
max,1.000000,1.000000,0.833333,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.851415,1.000000,1.000000,1.000000,0.998513,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
    model_sex = LGBMClassifier(n_estimators=1000, random_state=42, n_jobs=-1)    
    model_sex.fit(X_train_fold, y_train_fold_sex)
    y_val_pred_sex = model_sex.predict(X_val_fold)
    f1_sex = f1_score(y_val_fold_sex, y_val_pred_sex, average='macro')
    print(f"Fold Sex F1-score: {f1_sex:.4f}")

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 333, number of negative: 638
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.438736 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3084624
[LightGBM] [Info] Number of data points in the train set: 971, number of used features: 12112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.342945 -> initscore=-0.650196
[LightGBM] [Info] Start training from score -0.650196
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Initialize lists to store F1-scores for each fold
f1_scores_sex = []
f1_scores_adhd = []

# Define groups_train based on a column from your training data
groups_train = c_train['participant_id']

group_kfold = GroupKFold(n_splits=5)

for train_idx, val_idx in group_kfold.split(X_tr_df, y_adhd, groups_train):
    X_train_fold, X_val_fold = X_tr_df.iloc[train_idx], X_tr_df.iloc[val_idx]
    y_train_fold_adhd, y_val_fold_adhd = y_adhd.iloc[train_idx], y_adhd.iloc[val_idx]
    y_train_fold_sex, y_val_fold_sex = y_sex.iloc[train_idx], y_sex.iloc[val_idx]
    
    # Train and evaluate model for Sex
    model_sex = LGBMClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
    model_sex.fit(X_train_fold, y_train_fold_sex)
    y_val_pred_sex = model_sex.predict(X_val_fold)
    f1_sex = f1_score(y_val_fold_sex, y_val_pred_sex, average='macro')
    f1_scores_sex.append(f1_sex)
    
    # Train and evaluate model for ADHD
    model_adhd = LGBMClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
    model_adhd.fit(X_train_fold, y_train_fold_adhd)
    y_val_pred_adhd = model_adhd.predict(X_val_fold)
    f1_adhd = f1_score(y_val_fold_adhd, y_val_pred_adhd, average='macro')
    f1_scores_adhd.append(f1_adhd)

# Calculate standard deviation for each fold
std_f1_sex = np.std(f1_scores_sex)
std_f1_adhd = np.std(f1_scores_adhd)

print(f"Standard Deviation of F1-scores for Sex: {std_f1_sex:.4f}")
print(f"Standard Deviation of F1-scores for ADHD: {std_f1_adhd:.4f}")

In [30]:
# Train and evaluate model for ADHD
model_adhd = LGBMClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
model_adhd.fit(X_train_fold, y_train_fold_adhd)
y_val_pred_adhd = model_adhd.predict(X_val_fold)
f1_adhd = f1_score(y_val_fold_adhd, y_val_pred_adhd, average='macro')
f1_scores_adhd.append(f1_adhd)

# Calculate standard deviation for each fold
std_f1_adhd = np.std(f1_scores_adhd)



c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 660, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.525335 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3084624
[LightGBM] [Info] Number of data points in the train set: 971, number of used features: 12112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.679712 -> initscore=0.752447
[LightGBM] [Info] Start training from score 0.752447
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

NameError: name 'f1_scores_adhd' is not defined

In [33]:
f1_scores_adhd = []

f1_scores_adhd.append(f1_adhd)

# Calculate standard deviation for each fold
std_f1_adhd = np.std(f1_scores_adhd)
print (f"ADHD F1-score: {f1_adhd:.4f} +/- {std_f1_adhd:.4f}")   
print (std_f1_adhd)

ADHD F1-score: 0.7833 +/- 0.0000
0.0


In [ ]:
    model_adhd = LGBMClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
    model_adhd.fit(X_train_fold, y_train_fold_adhd)
    y_val_pred_adhd = model_adhd.predict(X_val_fold)
    f1_adhd = f1_score(y_val_fold_adhd, y_val_pred_adhd, average='macro')
    print(f"Fold ADHD F1-score: {f1_adhd:.4f}")


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 660, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.546401 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3065324
[LightGBM] [Info] Number of data points in the train set: 971, number of used features: 12035
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.679712 -> initscore=0.752447
[LightGBM] [Info] Start training from score 0.752447
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

0.7056

In [ ]:
pred_sex = model_sex.predict(X_ts_df)
pred_adhd = model_adhd.predict(X_ts_df) 

xgboost with multioutput : Validation F1-score: 0.5189


feature_importances

LGBM

In [ ]:

"""from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeClassifier

model = MultiOutputClassifier(make_pipeline(
    ColumnTransformer([('imputer', SimpleImputer(), X_tr)],
                     remainder='passthrough',
                     verbose_feature_names_out=False),
    
    ColumnTransformer([('log', 
                       FunctionTransformer(np.log1p), log_features)],
                     remainder='passthrough'),
    
    MinMaxScaler(),
    
    RidgeClassifier(alpha=100)
))

model.fit(pd.DataFrame(X_tr, columns=features), y_tr)"""


"from sklearn.pipeline import make_pipeline\nfrom sklearn.preprocessing import FunctionTransformer\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.linear_model import RidgeClassifier\n\nmodel = MultiOutputClassifier(make_pipeline(\n    ColumnTransformer([('imputer', SimpleImputer(), X_tr)],\n                     remainder='passthrough',\n                     verbose_feature_names_out=False),\n    \n    ColumnTransformer([('log', \n                       FunctionTransformer(np.log1p), log_features)],\n                     remainder='passthrough'),\n    \n    MinMaxScaler(),\n    \n    RidgeClassifier(alpha=100)\n))\n\nmodel.fit(pd.DataFrame(X_tr, columns=features), y_tr)"

In [ ]:
""" import numpy as np
import pandas as pd
import multiprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

# Define models
ridge_model = MultiOutputClassifier(make_pipeline(
    ColumnTransformer([('imputer', SimpleImputer(), features)],
                     remainder='passthrough',
                     verbose_feature_names_out=False),
    
    ColumnTransformer([('log', FunctionTransformer(np.log1p), log_features)],
                     remainder='passthrough'),
    
    MinMaxScaler(),
    
    RidgeClassifier(alpha=100)
))

xgb_model = XGBClassifier(
    max_depth=3,
    n_estimators=1000,
    learning_rate=0.05,
    n_jobs=multiprocessing.cpu_count()
)

# Stacking model
stacked_model = MultiOutputClassifier(StackingClassifier(
    estimators=[('ridge', ridge_model), ('xgb', xgb_model)],
    final_estimator=RidgeClassifier(),  # Meta-classifier
    passthrough=True,  # Use original features in meta-model
    n_jobs=-1
))

# Train the stacked model
stacked_model.fit(pd.DataFrame(X_tr, columns=features), y_tr) """


" import numpy as np\nimport pandas as pd\nimport multiprocessing\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.preprocessing import FunctionTransformer, MinMaxScaler\nfrom sklearn.linear_model import RidgeClassifier\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.multioutput import MultiOutputClassifier\nfrom sklearn.ensemble import StackingClassifier\nfrom xgboost import XGBClassifier\n\n# Define models\nridge_model = MultiOutputClassifier(make_pipeline(\n    ColumnTransformer([('imputer', SimpleImputer(), features)],\n                     remainder='passthrough',\n                     verbose_feature_names_out=False),\n    \n    ColumnTransformer([('log', FunctionTransformer(np.log1p), log_features)],\n                     remainder='passthrough'),\n    \n    MinMaxScaler(),\n    \n    RidgeClassifier(alpha=100)\n))\n\nxgb_model = XGBClassifier(\n    max_depth=3,\n    n_estimators=1000,\n    learning_rate=0.05,\n    

Submission

In [ ]:
test_pred = test[['participant_id']].copy()
test_pred['ADHD_Outcome'] = pred_sex
test_pred['Sex_F'] = pred_adhd  

test_pred.to_csv(f"submission_{f1_sex:.4f}_{f1_adhd:.4f}.csv", index=False)
